In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import requests

In [ ]:
df = pd.read_parquet("sas_sarl_ouvertes.parquet")
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.dtypes

---

##### Traitement des colonnes Dates en datetime

In [ ]:
# # Liste des colonnes qui contiennent des dates
# date_cols = [
#     "Date de création de l'établissement",
#     "Date de la dernière mise à jour de l'établissement",
#     "Date du début de la période de l'établissement",
#     "Date de création de l'unité légale",
#     "Date de début de l'unité légale",
#     "Date de fermeture de l'unité légale"
# ]

# # Conversion en datetime
# for col in date_cols:
#     df[col] = pd.to_datetime(df[col], errors="coerce")

# # Vérifier
# df[date_cols].info()


In [ ]:
# 🔹 Créer la colonne finale
df["Date_fermeture_finale"] = df[["Date de fermeture de l'unité légale"]].max(axis=1)

# 🔹 Supprimer l'ancienne colonne
df.drop(columns=["Date de fermeture de l'unité légale"], inplace=True)

In [ ]:
df.head()

---

#### Calul des tranches d'effectif

In [ ]:
df["Tranche de l'effectif de l'unité légale"].unique()

In [ ]:
# Nettoyage des espaces éventuels
df["Tranche de l'effectif de l'unité légale"] = df["Tranche de l'effectif de l'unité légale"].str.strip()

# Mapping complet des tranches d'effectif à des valeurs numériques
mapping = {
    "0 salarié": 0,
    "Etablissement non employeur": 0,
    "1 ou 2 salariés": 1,
    "3 à 5 salariés": 3,
    "6 à 9 salariés": 6,
    "10 à 19 salariés": 10,
    "20 à 49 salariés": 20,
    "50 à 99 salariés": 50,
    "100 à 199 salariés": 100,
    "200 à 249 salariés": 200,
    "250 à 499 salariés": 250,
    "500 à 999 salariés": 500,
    "1 000 à 1 999 salariés": 1000
}

# Conversion avec map
df['Tranche_effectif_num'] = df["Tranche de l'effectif de l'unité légale"].map(mapping).astype('Int64')

# Vérification : quelles valeurs n'ont pas été mappées
valeurs_non_mappees = df[df['Tranche_effectif_num'].isna()]["Tranche de l'effectif de l'unité légale"].unique()
if len(valeurs_non_mappees) > 0:
    print("⚠️ Certaines valeurs n'ont pas été mappées :", valeurs_non_mappees)
else:
    print("✅ Toutes les valeurs ont été mappées correctement.")

# Vérification finale
df[["Tranche de l'effectif de l'unité légale", "Tranche_effectif_num"]].head(10)


In [ ]:
# dropper la colonne originale
df=df.drop(columns=['Tranche de l\'effectif de l\'unité légale'])

In [ ]:
df['Tranche_effectif_num'].value_counts()

##### Sortie des effectifs de plus de 49 salariés

In [ ]:
df_filtered = df[df["Tranche_effectif_num"] < 50].copy()
df_filtered.shape

In [ ]:
# Vérification
df_filtered['Tranche_effectif_num'].value_counts()

---

##### Calcul de l'âge estimé avec arrêt d'existence au 31/12/25 pour uniformisation

In [ ]:
# Colonnes de démarrage
cols_dates = [
    "Date de création de l'unité légale",
    "Date de début de l'unité légale",
    "Date de création de l'établissement",
    "Date du début de la période de l'établissement"
]

# Paramètres
borne_min = pd.Timestamp("1990-01-01") 
date_aberrante = pd.Timestamp("1900-01-01")
date_ref = pd.Timestamp("2025-12-31")

# --- Étape 1 : nettoyage des dates aberrantes ---
for col in cols_dates:
    df_filtered[col] = pd.to_datetime(df_filtered[col], errors="coerce")
    df_filtered.loc[df_filtered[col] == date_aberrante, col] = pd.NaT
    df_filtered.loc[df_filtered[col] < borne_min, col] = pd.NaT

# --- Étape 2 : calcul des âges en années ---
df_filtered["age_unite_legale"] = (date_ref - df_filtered["Date de création de l'unité légale"]).dt.days / 365.25
df_filtered["age_debut_unite"] = (date_ref - df_filtered["Date de début de l'unité légale"]).dt.days / 365.25
df_filtered["age_etablissement"] = (date_ref - df_filtered["Date de création de l'établissement"]).dt.days / 365.25

# --- Étape 3 : estimation unique de l'ancienneté ---

df_filtered["age_estime"] = df_filtered[["age_unite_legale", "age_debut_unite", "age_etablissement"]].max(axis=1)

# --- Étape 3b : convertir en entier ---
df_filtered["age_estime"] = df_filtered["age_estime"].round(0).astype("Int64")

# --- Étape 4 : indicateur de fiabilité ---
df_filtered["age_estime_incertain"] = df_filtered[cols_dates].isna().all(axis=1).astype(int)

# --- Étape 5 : suppression des colonnes brutes pour un dataset compact ---
df_filtered = df_filtered.drop(columns=cols_dates + ["age_unite_legale", "age_debut_unite", "age_etablissement"])

# --- Vérification rapide ---
df_filtered[["age_estime", "age_estime_incertain"]].head()

In [ ]:
df_filtered['age_estime_incertain'].value_counts()

In [ ]:
df_filtered=df_filtered.drop(columns=['age_estime_incertain'])
df_filtered.head()

#### Visu de la distribution des âges

In [ ]:
# visu de la distribution
fig = px.box(
    df_filtered,
    y="age_estime",
    title="Boxplot de l'age des sociétés"
)

fig.show()

##### Plafonnement aux outliers

In [ ]:
# Supprimer les sociétés de plus de 28 ans
df_filtered =df_filtered[(df_filtered['age_estime'] < 36)].copy()
df_filtered.head()

#### Visu de la répartition des âges et effectifs

In [ ]:
# Comptage du nombre de sociétés par âge
age_counts = df_filtered["age_estime"].value_counts().sort_index().reset_index()
age_counts.columns = ["age", "nombre_societes"]

# Barplot interactif
fig = px.bar(
    age_counts,
    x="age",
    y="nombre_societes",
    title="Répartition du nombre de sociétés existantes par âge",
    labels={"age": "Âge (années)", "nombre_societes": "Nombre de sociétés"},
    text="nombre_societes",
    height=500,
    width=1200
)

# Affichage du nombre sur les barres
fig.update_traces(textposition="outside")

# Affichage
fig.show()

In [ ]:
# Grouper par âge et tranche d'effectif
df_counts = (
    df_filtered.groupby(['age_estime', 'Tranche_effectif_num'])
      .size()
      .reset_index(name='nombre_societes')
)

# Ordre croissant des tranches d'effectif
effectif_order = sorted(df_counts["Tranche_effectif_num"].unique())

# Scatter interactif avec échelle de couleurs adaptée
fig = px.scatter(
    df_counts,
    x='age_estime',
    y='Tranche_effectif_num',
    size='nombre_societes',
    color='nombre_societes',              
    color_continuous_scale='Viridis',       
    hover_data=['nombre_societes'],
    title="Nombre de sociétés existantes par âge et tranche d'effectif",
    labels={
        'age_estime': "Âge estimé (années)",
        'Tranche_effectif_num': "Tranche d'effectif",
        'nombre_societes': "Nombre de sociétés"
    },
    size_max=40,
    height=600,
    width=1200
)

fig.update_yaxes(type='category', categoryorder='array', categoryarray=effectif_order)

fig.show()

---

#### Traitement de l'existence des structures

In [ ]:
df_filtered["Etat administratif de l\'unité légale"].value_counts()

In [ ]:
df_fermee = df_filtered[df_filtered["Etat administratif de l'unité légale"] == "Cessée"]
df_fermee

In [ ]:
# Structure inactive, droppée

df_filtered = df_filtered[df_filtered["Etat administratif de l'unité légale"] != "Cessée"]

In [ ]:
# Vérification finale
df_filtered["Etat administratif de l\'unité légale"].value_counts()

In [ ]:
# Drop de la colonne
df_filtered = df_filtered.drop(columns=["Etat administratif de l'unité légale"])
df_filtered.head()

---

##### Traitement du Code postal

In [ ]:
df_filtered.dtypes

In [ ]:
# Correction du chaînage et fermeture de la parenthèse
df_filtered["Date de la dernière mise à jour de l'établissement"] = (
    pd.to_datetime(
        df_filtered["Date de la dernière mise à jour de l'établissement"],
        errors="coerce",
        utc=True       
    )
    .dt.tz_convert(None)
    .dt.normalize()      
)

# Affichage des premières lignes
print(df_filtered.head())

---

In [ ]:
df_filtered["Date de la dernière mise à jour de l\'établissement"].value_counts(ascending=True)

##### Les dates de dernière mise à jour ne sont pas aberrante, pas de drops de lignes, mais drop de la colonne devenue inutile.

In [ ]:
df_filtered.drop(
    columns=["Date de la dernière mise à jour de l'établissement"],
    inplace=True
)
df_filtered.head()

---

#### Visu de la répartition selon la catégorie juridique

In [ ]:
df_filtered.columns.to_list()

In [ ]:
# Comptage par nature juridique
counts = df_filtered["Nature juridique de l'unité légale"].value_counts()

# Pie chart
plt.figure()
plt.pie(
    counts,
    labels=counts.index,
    autopct="%1.1f%%",
    startangle=90
)
plt.title("Répartition selon la nature juridique")
plt.axis("equal")
plt.show()

##### Drop de la colonne catégorielle de la catégorie, la colonne numérique étant conservée et évitant les doublons

In [ ]:
df_filtered.drop(
    columns=["Nature juridique de l'unité légale"],
    inplace=True
)
df_filtered.head()

---

#### Traitement de la géolocalisation

In [ ]:
df_map = df_filtered.copy()

coords = df_map["Géolocalisation de l'établissement"].str.split(",", expand=True)
df_map["latitude"] = coords[0].astype(float)
df_map["longitude"] = coords[1].astype(float)

df_map=df_map.drop(columns=['Géolocalisation de l\'établissement'])

df_map.head()

---

##### Heatmap des sociétés ouvertes

In [ ]:
df_dep = (
    df_map.groupby("Code du département de l'établissement")
      .size()
      .reset_index(name="nb_etablissements")
)
df_dep.head()


In [ ]:
# 1️⃣ Charger le GeoJSON des départements depuis le web
url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
r = requests.get(url)
geojson_dep = r.json()

# 2️⃣ Préparer les données : compter les établissements par département
df_dep = df.groupby("Code du département de l'établissement").size().reset_index(name="nb_etablissements")

# 3️⃣ S'assurer que les codes sont bien au format 2 caractères
df_dep["Code du département de l'établissement"] = df_dep["Code du département de l'établissement"].astype(str).str.zfill(2)

# 4️⃣ Créer la carte choroplèthe
fig = px.choropleth(
    df_dep,
    geojson=geojson_dep,
    locations="Code du département de l'établissement",
    featureidkey="properties.code",
    color="nb_etablissements",
    color_continuous_scale="Reds",
    scope="europe",
    labels={"nb_etablissements": "Nombre d'établissements"},
    title="Répartition des établissements par département",
    width=800,
)

# 5️⃣ Ajuster l'affichage
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0, "t":40, "l":0, "b":0})

fig.show()

---

##### Visu des activités

In [ ]:
ape_counts=df_map["Activité principale de l'unité légale"].value_counts()
ape_percent = ape_counts / ape_counts.sum() * 100

ape_percent.head(30)

In [ ]:
top_30 = ape_counts.sort_values(ascending=False).head(30)

plt.figure(figsize=(10,8))
plt.barh(top_30.index[::-1], top_30.values[::-1], color='skyblue')
plt.xlabel("Nombre d'établissements")
plt.ylabel("Code APE")
plt.title("Top 30 des codes APE les plus fréquents des sociétés ouvertes")
plt.tight_layout()
plt.show()

##### Import des codes APE officiels pour analyse des secteurs

In [ ]:
ape=pd.read_csv('codes_ape.csv')
ape.head()

In [ ]:
# --- Étape 1 : créer une colonne "ape_section" avec les 2 premiers chiffres ---

df_map["Activité principale de l'unité légale"] = df_map["Activité principale de l'unité légale"].astype(str)

# On prend les 2 premiers chiffres du code APE
df_map["ape_section"] = df_map["Activité principale de l'unité légale"].str[:2]

# --- Étape 2 : préparer le mapping avec ton fichier ape ---

ape["code_ape"] = ape["code_ape"].astype(str)

# On ne garde que les sections de 2 caractères pour le mapping
ape_sections = ape[ape["code_ape"].str.len() == 2][["code_ape", "libelle_ape"]]

# --- Étape 3 : fusionner pour ajouter le libellé de la section ---
df_map = df_map.merge(ape_sections, how="left", left_on="ape_section", right_on="code_ape")

# On peut renommer la colonne si nécessaire
df_map = df_map.rename(columns={"libelle_ape": "libelle_section_ape"})

# --- Vérification ---
df_map[["Activité principale de l'unité légale", "ape_section", "libelle_section_ape"]].head(10)

In [ ]:
print(df_map["libelle_section_ape"].isna().sum())

##### Nouveaux visus selon regroupement par secteur d'activité

In [ ]:
# --- Étape 1 : compter le nombre d'entreprises par section APE ---
section_counts = df_map.groupby("ape_section") \
                   .size() \
                   .reset_index(name="nombre_entreprises")

# --- Étape 2 : récupérer le libellé correspondant pour chaque section ---

libelles = df_map.groupby("ape_section")["libelle_section_ape"].first().reset_index()
section_counts = section_counts.merge(libelles, on="ape_section")

# --- Étape 3 : trier par ordre décroissant et prendre les 20 premières ---
section_counts = section_counts.sort_values("nombre_entreprises", ascending=False).head(20)

# --- Étape 4 bis : visualiser avec un barplot interactif avec les plus grandes valeurs en haut ---
fig = px.bar(
    section_counts,
    y="libelle_section_ape",
    x="nombre_entreprises",
    title="Top 20 des sections APE par nombre d'entreprises ouvertes",
    labels={"libelle_section_ape": "Secteur d'activité", "nombre_entreprises": "Nombre d'entreprises"},
    text="nombre_entreprises",
    height=600,
    width=1600,
    category_orders={"libelle_section_ape": section_counts.sort_values("nombre_entreprises", ascending=False)["libelle_section_ape"].tolist()}
)

# Pour rendre le texte lisible
fig.update_traces(textposition="outside")

# Ajustement des marges pour les labels longs
fig.update_layout(xaxis_tickangle=-45, margin=dict(l=200))

fig.show()

##### Selon age et effectif

In [ ]:
# --- Étape 1 : filtrer uniquement les 30 sections les plus nombreuses ---
top_sections = section_counts['ape_section'].tolist()
df_top = df_map[df_map['ape_section'].isin(top_sections)]

# --- Étape 2 : calculer l'âge moyen par section ---
age_moyenne = df_top.groupby(['ape_section', 'libelle_section_ape'])['age_estime'].mean().reset_index()
age_moyenne = age_moyenne.rename(columns={'age_estime': 'age_moyen'})

# --- Étape 3 : fusionner avec le nombre d'entreprises pour le graphique ---
age_moyenne = age_moyenne.merge(section_counts[['ape_section', 'nombre_entreprises']], on='ape_section')

# --- Étape 4 : barplot interactif avec palette rouge → vert → bleu ---
fig = px.bar(
    age_moyenne,
    y='libelle_section_ape',
    x='nombre_entreprises',
    color='age_moyen',                      
    text='age_moyen',
    title="Top 20 sections APE sociétés ouvertes : nombre d'entreprises et âge moyen",
    labels={
        'libelle_section_ape': "Secteur d'activité",
        'nombre_entreprises': "Nombre d'entreprises",
        'age_moyen': "Âge moyen (années)"
    },
    height=600,
    width=1500,
    category_orders={"libelle_section_ape": age_moyenne.sort_values("nombre_entreprises", ascending=False)["libelle_section_ape"].tolist()},
    color_continuous_scale=['red','green','blue']
)

# Rendre le texte lisible
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')

# Ajuster les marges pour les labels longs
fig.update_layout(xaxis_tickangle=-45, margin=dict(l=200))

fig.show()

#### Exploration durée moyenne selon les 20 secteurs les plus représentés

In [ ]:
# --- Étape 1 : calculer la moyenne du nombre de périodes par section ---
moyennes_age = df_map.groupby("ape_section")["age_estime"] \
                      .mean() \
                      .round(2) \
                      .reset_index(name="moyenne_age")

# --- Étape 2 : ne garder que les 20 sections les plus fréquentes ---
top20_sections = section_counts["ape_section"]
moyennes_top20 = moyennes_age[moyennes_age["ape_section"].isin(top20_sections)]

# --- Étape 3 : ajouter le libellé pour plus de lisibilité ---
moyennes_top20 = moyennes_top20.merge(
    section_counts[["ape_section", "libelle_section_ape"]],
    on="ape_section"
)

# --- Étape 4 : afficher le résultat trié par moyenne décroissante ---
moyennes_top20 = moyennes_top20.sort_values("moyenne_age", ascending=False)
moyennes_top20

In [ ]:
# --- On trie les données par moyenne décroissante ---
moyennes_top20_sorted = moyennes_top20.sort_values("moyenne_age", ascending=False)

# --- Graphique ---
fig = px.bar(
    moyennes_top20_sorted,
    y="libelle_section_ape",
    x="moyenne_age",
    title="Moyenne du nombre d'années par secteur (Top 20)",
    labels={
        "libelle_section_ape": "Secteur d'activité",
        "moyenne_age": "Moyenne du nombre d'années"
    },
    text="moyenne_age",
    height=600,
    width=1700,
    category_orders={"libelle_section_ape": moyennes_top20_sorted["libelle_section_ape"].tolist()}
)

# Texte à l'extérieur et ajustement des marges
fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_tickangle=-45,
    margin=dict(l=200),
    xaxis=dict(range=[8, 17])
)

fig.show()

---

In [ ]:
df_map.head()

In [ ]:
df_map=df_map.drop(columns=['ape_section'])

df_map.head()

---

In [ ]:
df_map.shape

---

#### Rajout d'une colonne "statut" = 0 pour définir la non-fermeture

In [ ]:
df_map['fermeture']=0
df_map.head()

---

##### Traitement des Nan restants

In [ ]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df_map.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

In [ ]:
df_map = df_map.dropna()


In [ ]:
df_map.head()

In [ ]:
df_map.shape

In [ ]:
df_map.dtypes

---

In [ ]:
# Dernière vérification des codes de département
print(df_map["Code du département de l'établissement"].unique())

##### Export en parquet

In [ ]:
# df_map.to_parquet("dataset_open_final.parquet", index=False)